In [1]:
from_date = '20230320'
to_date = '20230326'
searching_word = 'finance OR banking OR investment OR economy'

In [2]:
import os
import pickle
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import time
import numpy

import re
import requests 
from bs4 import BeautifulSoup
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from newspaper import Article

import nltk
from keybert import KeyBERT

In [23]:
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import shutil

try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
except FileNotFoundError:
    pass

subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동


option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

C:\Users\NH\AppData\Local\Temp\ipykernel_29084\1447849451.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)


In [4]:
cwd = os.getcwd()
main_path = cwd + '/' + searching_word

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

createFolder(main_path)
createFolder(main_path + '/' + 'url')
createFolder(main_path + '/' + 'news_backup')
createFolder(main_path + '/' + 'news')
createFolder(main_path + '/' + 'keyword')

In [25]:
# 1 키워드 검색 뉴스 url 수집
def page2url(searching_word,date):
    searching_word = '{}'.format(searching_word)
    urls = []
    for start in range(0, 360, 10): ### 하루에 관련 기사를 최대 얼마나 뽑을지 설정.
        main_url = 'https://www.google.co.kr/search?q={}&tbs=cdr:1,cd_min:{},cd_max:{}&tbm=nws&ei=dPP-Yu_eCJLL-Qb55bvQDA&start={}&sa=N&ved=2ahUKEwjv6Lvy69H5AhWSZd4KHfnyDso4ChDy0wN6BAgBEDk&biw=1536&bih=754&dpr=1.25'.format(searching_word,date,date,start)
        driver.get(url=main_url)
        elements = driver.find_elements(By.TAG_NAME, 'a')
        lnks = []
        for lnk in elements:
            lnk = str(lnk.get_attribute('href'))
            if 'google' not in lnk and lnk != 'None':
                lnks.append(lnk)
        if len(lnks) == 0:
            print(date, ' Url 수집 완료')
            break
        urls.extend(lnks)
        rand_value =random.uniform(4, 10)
        time.sleep(rand_value)
        
    return urls

datelist = pd.date_range(start=from_date, end=to_date).tolist()
dtlst = []
for d_t in datelist:
    d_t = str(d_t)[0:-9]
    d = datetime.strptime(d_t, '%Y-%m-%d')
    d = d.strftime('%m/%d/%Y')
    d = d[0].replace('0','') + d[1:]
    d = d[:-7] + d[-7].replace('0','') + d[-6:]
    dtlst.append(d)
    
    
Urls = dict()
for date in tqdm(dtlst):
    print(date,'Url 수집 시작')
    urls = page2url(searching_word,date)
    Urls[date] = urls
    
name = "Urls{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

with open(main_path + '/' + 'url/' + name,'wb') as f:
    pickle.dump(Urls,f)


  0%|                                                                                            | 0/7 [00:00<?, ?it/s]

3/17/2023 Url 수집 시작


 14%|███████████▊                                                                       | 1/7 [02:44<16:29, 164.90s/it]

3/17/2023  Url 수집 완료
3/18/2023 Url 수집 시작


 29%|███████████████████████▋                                                           | 2/7 [05:31<13:50, 166.15s/it]

...

3/21/2023  Url 수집 완료
3/22/2023 Url 수집 시작


 86%|███████████████████████████████████████████████████████████████████████▏           | 6/7 [17:38<03:04, 184.60s/it]

3/22/2023  Url 수집 완료
3/23/2023 Url 수집 시작


100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [21:11<00:00, 181.68s/it]

3/23/2023  Url 수집 완료


In [6]:
name = "Urls{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')
# 2  수집된 url로 news 크롤링 
with open(main_path + '/' + 'url/' + name, 'rb') as f:
    data = pickle.load(f)

# start = 0
for date in list(data):
    print(date, '크롤링 시작')
    cnt = 0
    tmp = []
    urls = []
    for idx,url in tqdm(enumerate(data[date])):
        try:
            article = Article(url)
            article.download()
            article.parse()
            v = article.text
            tmp.append(v)
            urls.append(url)
            cnt += 1
        except:
            pass
            tmp.append("null")
            urls.append(url)
            print(url)
    print("총 {}건 중 {} 건 크롤링 성공".format(idx+1,cnt))
    data[date] = [tmp,[urls]]
    date = date.replace('/','-')
    with open(main_path + '/' + 'news_backup/' + 'news-' + date + '.pickle','wb') as f:
        pickle.dump(data,f)
    print(date,' 크롤링 완료')

3/20/2023 크롤링 시작


37it [00:35,  1.35it/s]

https://www.forbes.com/sites/forbesbusinesscouncil/2023/03/20/learning-from-the-evolution-of-open-banking/


42it [00:40,  1.08s/it]

https://www.forbes.com/sites/forbesbusinesscouncil/2023/03/20/the-future-of-ai-in-banking/


...

https://www.business-standard.com/opinion/columns/g20-must-prioritise-climate-finance-123032600249_1.html


185it [03:55,  1.02s/it]

https://financialpost.com/news/economy/companies-invest-now-recession-coming


211it [04:41,  1.34s/it]

총 211건 중 195 건 크롤링 성공
3-26-2023  크롤링 완료


In [7]:
data = pd.DataFrame(data).T

data = data.rename({0:"news",1:"url"},axis= 1)

name = "News{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

with open(main_path + '/' + 'news/' + name,'wb') as f:
    pickle.dump(data,f)

In [8]:
def doc2key(cleaned_content):
    
    kw_model = KeyBERT(model='all-MiniLM-L6-v2')
    keywords = kw_model.extract_keywords(cleaned_content)
    
    n2_kwd = kw_model.extract_keywords(cleaned_content, keyphrase_ngram_range=(2, 2), stop_words='english',
                                  use_mmr=True, diversity=0.7, top_n=5)

    n1_kwd = kw_model.extract_keywords(cleaned_content, keyphrase_ngram_range=(1, 1), stop_words='english',
                                  use_mmr=True, diversity=0.7, top_n=45)
    for idx,i in enumerate(n2_kwd):
        n2_kwd[idx] = i[0]
    for idx,i in enumerate(n1_kwd):
        n1_kwd[idx] = i[0]  

    n1_kwd.extend(n2_kwd)
    kwd = n1_kwd
    return kwd

In [9]:
def makeKeywordLsit(data,nn_jj = False):
    kwd_list = []
    for docs in tqdm(data['news']):
        kwds = []
        for doc in tqdm(docs):
            try:
                cleaned_content = re.sub(r'[^\.\?\!\w\d\s]','',doc) # 문장단위로 끊기
                cleaned_content = cleaned_content.replace('\n',' ')
                cleaned_content = cleaned_content.lower()
                kwd = doc2key(cleaned_content)
                if nn_jj == True:
                    tokens_pos = nltk.pos_tag(kwd)
                    kwd_nn_jj = []
                    for word, pos in tokens_pos:
                        if 'NN' in pos or 'JJ' in pos:
                            kwd_nn_jj.append(word)
                    kwds.append(kwd_nn_jj)
                else:
                    kwds.append(kwd)
            except:
                print(doc)
        kwd_list.append(kwds)
    return kwd_list

In [10]:
# 데이터불러오기

name = "News{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')

with open(main_path + '/' + 'news/' + name, 'rb') as f:
    data = pickle.load(f)

In [11]:
# 전처리
data = data.reset_index()
data = data.rename({"index":"date"},axis = 1)

idx = []
for x in data['date']:
    tmp = x.split('/')
    if len(tmp[0]) == 1:
        tmp[0] = '0'+tmp[0]
    if len(tmp[1]) == 1:
        tmp[1] = '0'+tmp[1]
    tmp = tmp[2] + tmp[0] + tmp[1]
    idx.append(''.join(tmp))
    
data['date'] = idx
data = data.rename({'date':'일자'},axis = 1)

In [12]:
# 키워드 추출
kwd_list = makeKeywordLsit(data, nn_jj = True)

 30%|████████████████████████                                                         | 64/215 [03:03<04:59,  1.98s/it]


 60%|████████████████████████████████████████████████▎                               | 130/215 [14:32<03:44,  2.64s/it]


 91%|████████████████████████████████████████████████████████████████████████▉       | 196/215 [17:30<01:01,  3.26s/it]


 21%|████████████████▋                                                                | 45/218 [02:40<11:30,  3.99s/it]


 51%|████████████████████████████████████████▋                                       | 111/218 [06:20<07:35,  4.26s/it]


...

 81%|████████████████████████████████████████████████████████████████▊               | 179/221 [08:45<02:11,  3.13s/it]


 10%|████████▍                                                                        | 22/211 [01:23<14:13,  4.52s/it]


 42%|█████████████████████████████████▊                                               | 88/211 [04:20<05:15,  2.56s/it]


 73%|██████████████████████████████████████████████████████████▍                     | 154/211 [07:38<02:24,  2.53s/it]


100%|█████████████████████████████████████████████████████████████████████████████████| 7/7 [1:26:23<00:00, 740.45s/it]


In [13]:
# 데이터 추가
data['키워드'] = kwd_list

In [14]:
# 저장
name = "Keyword{}-{}.pickle".format(from_date,to_date)
name = name.replace('/','.')
with open(main_path + '/' + 'keyword/' + name,'wb') as f:
    pickle.dump(data,f)